# Investigating and cleaning "Ticket validation logs" (Validation_by_stops folder in raw_data)

a) Is there missing data that needs imputation?

b) Are the names of bus lines correct?

c) Are the bus stops correct?

In [1]:
import glob 
import pandas as pd

In [2]:
files = glob.glob("c:/users/henry chapman/Documents/Coding/Data_science/Project_final/Raw_data/Validations_by_stops/*.csv")
dfs = []
for file in files:
    df = pd.read_csv(file, 
                     skiprows = 2,
                     on_bad_lines = "skip",
                     skipfooter = 1,
                     engine = "python")
    dfs.append(df)


Data = pd.concat(dfs, ignore_index = True)

translation = {
    "Kuupäev": "Date",
    "Organisatsioon": "Organisation",
    "Liin": "Line",
    "Väljumin": "Departure_ID",
    "Reis": "Trip",
    "Algus": "Start",
    "Peatus": "Stop",
    "Peatuse ": "Stop_ID",
    "Kell": "Time",
    "Valideerimi": "Validation_Count"
}

Data.rename(columns = translation, inplace = True)

In [3]:
Filtered_validations = Data.copy()

In [4]:
Filtered_validations.head()

,Date,Organisation,Line,Departure_ID,Trip,Start,Stop,Stop_ID,Time,Validation_Count
0,01.01.16,SEBE AS,1,80683.0,Nõlvaku,10:00,Teaduspark,7820277-1,10:01,1.0
1,01.01.16,SEBE AS,1,80683.0,Nõlvaku,10:00,Kesklinn,7820088-1,10:12,3.0
2,01.01.16,SEBE AS,1,80739.0,FI,10:01,Nõlvaku,7820165-1,10:01,2.0
3,01.01.16,SEBE AS,1,80739.0,FI,10:01,Mõisavahe,7820155-1,10:02,1.0
4,01.01.16,SEBE AS,1,80739.0,FI,10:01,Annelinna keskus,7820014-1,10:04,3.0


### Adding in pre 2019 / post 2019 column 

In [5]:
Filtered_validations["Date"] = pd.to_datetime(Filtered_validations["Date"], format = "%d.%m.%y")

real_switch = pd.Timestamp("2019-07-01")

Filtered_validations["Pre_2019"] = Filtered_validations["Date"] < real_switch

Filtered_validations.head()

,Date,Organisation,Line,Departure_ID,Trip,Start,Stop,Stop_ID,Time,Validation_Count,Pre_2019
0,2016-01-01,SEBE AS,1,80683.0,Nõlvaku,10:00,Teaduspark,7820277-1,10:01,1.0,True
1,2016-01-01,SEBE AS,1,80683.0,Nõlvaku,10:00,Kesklinn,7820088-1,10:12,3.0,True
2,2016-01-01,SEBE AS,1,80739.0,FI,10:01,Nõlvaku,7820165-1,10:01,2.0,True
3,2016-01-01,SEBE AS,1,80739.0,FI,10:01,Mõisavahe,7820155-1,10:02,1.0,True
4,2016-01-01,SEBE AS,1,80739.0,FI,10:01,Annelinna keskus,7820014-1,10:04,3.0,True


## Imputation of Missing times 

In [6]:
# looking at NaNs
Filtered_validations.isna().sum()

Date                 3741
Organisation         3741
Line                 3869
Departure_ID         3869
Trip                 3870
Start                3870
Stop                    1
Stop_ID              3743
Time                64510
Validation_Count     3742
Pre_2019                0
dtype: int64

In [7]:
# Filling with the previous time point - this is for identical stops - so the times are perfect - there is no interpolation 
Filtered_validations["Time"] = Filtered_validations.groupby(['Departure_ID', 'Start', 'Stop_ID', "Pre_2019"])['Time'].ffill()

In [8]:
# Filling with the afterwards time point 
Filtered_validations["Time"] = Filtered_validations.groupby(['Departure_ID', 'Start', 'Stop_ID', "Pre_2019"])['Time'].bfill()

In [9]:
Filtered_validations.isna().sum()

Date                 3741
Organisation         3741
Line                 3869
Departure_ID         3869
Trip                 3870
Start                3870
Stop                    1
Stop_ID              3743
Time                12672
Validation_Count     3742
Pre_2019                0
dtype: int64

In [10]:
# Now I perform interpolation by not grouping by Stop_ID - thus, simply taking the value of the bus stop before it, then after it
Filtered_validations["Time"] = Filtered_validations.groupby(['Departure_ID', 'Start', "Pre_2019"])['Time'].ffill()
Filtered_validations["Time"] = Filtered_validations.groupby(['Departure_ID', 'Start', "Pre_2019"])['Time'].bfill()

In [11]:
Filtered_validations.isna().sum()

Date                3741
Organisation        3741
Line                3869
Departure_ID        3869
Trip                3870
Start               3870
Stop                   1
Stop_ID             3743
Time                9455
Validation_Count    3742
Pre_2019               0
dtype: int64

In [12]:
# This is as much itnerpolation possible without 1. requiring quite high compute or relying on poor interpolation 
# 5700 invalid entries out of 30 million is reasonable - provided that they are not sqewed for one aprtiucalr bus stop / bus line 

In [13]:
Filtered_validations["Time"].dropna(inplace = True)

In [14]:
Filtered_validations["Time"] = Filtered_validations["Time"].astype(str)

# Combine Date and Time into a single datetime column
Filtered_validations["DateTime"] = pd.to_datetime(
    Filtered_validations["Date"].astype(str) + " " + Filtered_validations["Time"].astype(str),
    errors='coerce'  # invalid entries will become NaT
)



### Fixing incorrect Line IDs

Multiple names for the same bus line are present and needs fixing

In [15]:
Filtered_validations["Line"].unique()

array(['1', '10', '14', '15', '17', '18', '2', '20', '21', '22', '3', '4',
       '5', '6', '7', '8', '9', '16', '11', '12', '13', '16A', '19', '24',
       '26', '27', nan, '28', '62B', 'ATB', '69', '25', '69A', 'E1',
       'Go_12', 'Go_11', 'G1', 'G10', 'G11', 'G12', 'G13', 'G2', 'G3',
       'G4', 'G6', 'G7', 'G8', 'G9', 'G9A', 'G5', '9A', '09:43', '31R',
       '31'], dtype=object)

In [16]:
import numpy as np
# Unified line collapse dictionary
line_collapse_dict = {
    # Lettered variants
    '16A': '16',
    '69A': '69',
    '62B': '62',

    # G-prefixed routes (strip G)
    'G1': '1', 'G2': '2', 'G3': '3', 'G4': '4', 'G5': '5', 'G6': '6', 
    'G7': '7', 'G8': '8', 'G9': '9', 'G9A': '9A', 'G10': '10', 
    'G11': '11', 'G12': '12', 'G13': '13',

    # Go-prefixed routes
    'Go_11': '11',
    'Go_12': '12',

    # Numeric lines (float → str)
    1.0: "1", 2.0: "2", 3.0: "3", 4.0: "4", 5.0: "5", 6.0: "6", 7.0: "7",
    8.0: "8", 9.0: "9", 10.0: "10", 11.0: "11", 12.0: "12", 13.0: "13",
    21.0: "21", 22.0: "22", 25.0: "25",

    # Special lines
    'E1': 'E1',

    # Missing or irrelevant values
    '09:43': None,
    'nan': None
}

# Make everything a string, keep NaN as np.nan
Filtered_validations["Line"] = Filtered_validations["Line"].apply(lambda x: np.nan if pd.isna(x) else str(x))

# Apply collp[asing dictionary
Filtered_validations["Line_clean"] = Filtered_validations["Line"].replace(line_collapse_dict)



## Exporting main cleaned file

In [17]:
# Neccasary colunms 

keep_cols = ["DateTime", "Line_clean", "Stop_ID", "Pre_2019", "Validation_Count"]

Final = Filtered_validations[keep_cols]

Final.to_csv("c:/users/henry chapman/Documents/Coding/Data_science/Project_final/Output/1_Compiling_data/Pipe4/Ticket_validations.csv", index = False)

## Exporting unique stops IDS and their temporal existence

Given that the GTSF data base was updated in 2019 to reflect the change in the bus network - I will use this as the reference for which bus stops existed before and after 2019

In [18]:
Unique_stops = Final[["Stop_ID", "Pre_2019"]].drop_duplicates()
Unique_stops.to_csv("c:/users/henry chapman/Documents/Coding/Data_science/Project_final/Output/1_Compiling_data/Pipe4/Unique_stops.csv", index = False)
Unique_stops.head()

,Stop_ID,Pre_2019
0,7820277-1,True
1,7820088-1,True
2,7820165-1,True
3,7820155-1,True
4,7820014-1,True
